In [1]:
import requests
import urllib
from bs4 import BeautifulSoup
import string
import re
import pandas as pd


In [2]:
thomasnet_url = 'https://www.thomasnet.com'
# base_url = 'https://www.thomasnet.com/nsearch.html?cov=NA&heading=45340403&what=Machining&pg='
#page_letter = [string.ascii_uppercase[i] for i in range(1,2) ]
# page_letter = ['D']

In [3]:

def MainPage(mainContent):
    soup = BeautifulSoup(mainContent,'html.parser')
    tags = soup('a')
    #print(tags)
#b = soup.find('div', style = 'border-bottom: 1px solid #cecece; margin-bottom: 18px; padding: 0 0 12px 36px;')
#for i in b.find_all('a'):
    #print(i.get('href'))
    links=[]
    for tag in tags:
        if re.findall("^/browsecompanies\.html\?letter\=[A-Z0-9].+",tag.get('href',None)):
            links.append(re.findall("^/browsecompanies\.html\?letter\=[A-Z0-9].+",tag.get('href',None))[0])

    sub_page_urls = list(set(links))
    return sub_page_urls

In [4]:

def SubPage(subContent):
    soup = BeautifulSoup(subContent,'html.parser')
    tags = soup('a')
    links=[]
    for tag in tags:
        if re.findall("^/profile.+",str(tag.get('href',None))):
            links.append(re.findall("^/profile.+",tag.get('href',None))[0])
    manufacturer_profile_urls = list(set(links))
    
    return manufacturer_profile_urls


In [5]:


def ManufacturerPage(manufacturer_profile_url):
    manufacturerContent = urllib.request.urlopen(thomasnet_url+manufacturer_profile_url).read()
    soup = BeautifulSoup(manufacturerContent,'html.parser')
    company_state= []
    company_address = []
    company_phone_tag =[]
    company_tag = soup.find('div', class_='codetail' )
    #company_address = company_tag.find('p', class_ = 'addrline').get_text()
    #company_address = company_tag.find('p', class_ = 'addrline').contents[0][:-2]
    if company_tag.find('p', class_ = 'addrline'):
        #print(company_tag.find('p', class_ = 'addrline').get_text().split('|')[0])
         
        if (company_tag.find('p', class_ = 'addrline').get_text().split('|')[0]) == 'map':
            company_address = ''
            company_state = ''
        else :
            company_address = company_tag.find('p', class_ = 'addrline').get_text().split('|')[0]
            #print(company_address, type(company_address))
            company_state = company_address.split(', ')[1].split(' ')[0]
        
    
    supplier_data = company_tag.find('h1')
    supplier_name = supplier_data.get_text()
    
    company_tag = soup.find('div', class_='codetail' )
    company_phone_tag = company_tag.find('p', class_ = 'phoneline')
    
    supplier_number = []
    supplier_url = []
    if company_phone_tag:
        supplier_number = company_phone_tag.find('span').get_text()
        if company_phone_tag.find('a', class_ = "primary"):
            supplier_url = company_phone_tag.find('a', class_ = "primary").get('href')      
    
    #for web_address in company_phone_tag('a'):
     #   supplier_url = []
      #  if web_address.get('data-advertiser_link'):
       #     supplier_url = web_address.get('data-advertiser_link')
    #supplier_url = company_phone_tag.find('a', class_ = "primary").get('href')
    company_description_tag = soup.find('div', id= 'copro_about')
    supplier_description = []
    
    if company_description_tag:
        supplier_description = company_description_tag.get_text()
    
    #---------------Employees---------------------------------
    
    employees_tag = soup.find(class_ = 'bdcol2 match-height').find_all(class_ = 'bizdetail')
    employees_number = ''
    for tag in employees_tag:
        if tag.find(class_='label'):
            if tag.find(class_='label').text == 'No of Employees:':
                employees_number = str(tag.find('li').get_text())
        
    #------------- Capabilities----------------------------------------
    Capabilities_url = soup.find(class_="submenu").find_all('a')[1].get('href')
    CapabilitiesContent = urllib.request.urlopen(thomasnet_url+Capabilities_url).read()
    soup1 = BeautifulSoup(CapabilitiesContent,'html.parser')
    
    Capabilities_list= soup1.find_all(class_= 'expand show')
    
    supplier_capabilities_detailed = []
    supplier_capabilities_overall = []
    Casting = []
    for element in Capabilities_list:  
        #print('1')
        supplier_capabilities_detailed.append(element.text)
        supplier_capabilities_overall.append(element.a.text)
        #print(element.get_text()+'\n')
    #------------- Machining Milling Turning-----------------------------------------

    #for element1 in Capabilities_list:
        if re.findall("^Casting.*",element.a.text ):
            for element2 in element.find_all('li'):
                Casting.append(element2.text)
#         if re.findall("^Milling.*",element.a.text ):
#             for element2 in element.find_all('li'):
#                 Milling.append(element2.text)
#         if re.findall("^Turning.*",element.a.text ):
#             for element2 in element.find_all('li'):
#                 Turning.append(element2.text)
    
    #print(Capabilities)
    
    return company_address, supplier_name, supplier_number, supplier_url, supplier_description,supplier_capabilities_detailed, employees_number, Casting, supplier_capabilities_overall,company_state



In [6]:
# manufacturer_url = '/profile/00971394/the-federal-group-usa.html?cid=971394&cov=NA&heading=11421203&searchpos=22'


In [7]:
manufacturer_profile_urls = pd.read_csv('manufacturer_url_Casting.csv')
# print((manufacturer_profile_urls))

In [8]:


def main():
    #sub_page_url = []
    #for letter in page_letter:
     #   mainContent = urllib.request.urlopen(base_url+letter).read()

      #  sub_page_url += (MainPage(mainContent))

    #manufacturer_profile_urls = []
    #for sub_link in range(1,574):
     #   subContent = urllib.request.urlopen(base_url+str(sub_link)).read()
      #  manufacturer_profile_urls += SubPage(subContent)

    global Supplier
    Supplier = pd.DataFrame() 
   # print(len(manufacturer_profile_urls))
    #myfile = open('manufacturer_url', 'w')
    #mylist = map(lambda x:x+'\n', manufacturer_profile_urls)
    #myfile.writelines(mylist)
    #myfile.close()
    i = 0
    for manufacturer_url in manufacturer_profile_urls['Profile_URL']:
       
        i+=1
        print(i)
        print(manufacturer_url)
        Manufacturer_details= ManufacturerPage(manufacturer_url)
        supplier = {}
        supplier['Address'] = Manufacturer_details[0]
        supplier['Name']= Manufacturer_details[1]
        supplier['Number'] = Manufacturer_details[2]
        supplier['URL'] = Manufacturer_details[3]
        supplier['Description'] = Manufacturer_details[4][1:]
        supplier['Capabilities_Detailed'] = Manufacturer_details[5]
        supplier['Employees'] = Manufacturer_details[6]
        supplier['Casting_capabilities'] = Manufacturer_details[7]
        supplier['Capabilities_Overall'] = Manufacturer_details[8]
        supplier['Manufacturer_state'] = Manufacturer_details[9]

        #Supplier.append(supplier)
        Supplier = Supplier.append(supplier, ignore_index = True)
        Supplier= Supplier[['Name','Address','Manufacturer_state','Number','URL','Employees','Description','Casting_capabilities','Capabilities_Overall','Capabilities_Detailed']]
        
        #print(Supplier)

        Supplier.to_csv('Supplier-Casting.csv')
     







In [9]:
main()

1
/profile/00118955/custom-die-casting-co-inc.html?cid=118955&cov=NA&heading=11421203&searchpos=2059
2
/profile/20002371/wynn-danzur-marketing.html?cid=20002371&cov=NA&heading=11421203&searchpos=2058
3
/profile/00149422/alloymet.html?cid=149422&cov=NA&heading=11421203&searchpos=2061
4
/profile/01120418/rpm-engineering.html?cid=1120418&cov=NA&heading=11421203&searchpos=2066
5
/profile/30756014/dillon-precision-inc.html?cid=30756014&cov=NA&heading=11421203&searchpos=2053
6
/profile/01172839/wehner-foundry.html?cid=1172839&cov=NA&heading=11421203&searchpos=2064
7
/profile/00420175/westchester-casting-corp.html?cid=420175&cov=NA&heading=11421203&searchpos=2070
8
/profile/00032339/okeefe-casting-co.html?cid=32339&cov=NA&heading=11421203&searchpos=2060
9
/profile/00046656/nonferrous-heat-treating.html?cid=46656&cov=NA&heading=11421203&searchpos=2067
10
/profile/10034411/scot-hone-corp.html?cid=10034411&cov=NA&heading=11421203&searchpos=2063
11
/profile/01295864/dynamic-systems-international.

85
/profile/10084301/kroeger-na-co-inc.html?cid=10084301&cov=NA&heading=11421203&searchpos=2133
86
/profile/10040213/valley-pattern-mfg-inc.html?cid=10040213&cov=NA&heading=11421203&searchpos=2134
87
/profile/01115821/parks-bronze.html?cid=1115821&cov=NA&heading=11421203&searchpos=2175
88
/profile/10027212/tri-cast-bronze-ltd.html?cid=10027212&cov=NA&heading=11421203&searchpos=2154
89
/profile/00977938/fp-pla-tool.html?cid=977938&cov=NA&heading=11421203&searchpos=2165
90
/profile/01187686/hubbell-diecasting.html?cid=1187686&cov=NA&heading=11421203&searchpos=2167
91
/profile/00166857/lincoln-die-casting-inc.html?cid=166857&cov=NA&heading=11421203&searchpos=2153
92
/profile/01175562/idaho-falls-foundry-machine.html?cid=1175562&cov=NA&heading=11421203&searchpos=2160
93
/profile/30264208/wise-associates-inc.html?cid=30264208&cov=NA&heading=11421203&searchpos=2151
94
/profile/00078928/quality-cast-metals-inc.html?cid=78928&cov=NA&heading=11421203&searchpos=2174
95
/profile/00456636/badger-p

168
/profile/10018859/hubbard-casting-co-inc.html?cid=10018859&cov=NA&heading=11421203&searchpos=2235
169
/profile/00138442/new-jersey-shell-casting-corp.html?cid=138442&cov=NA&heading=11421203&searchpos=2250
170
/profile/20077160/versa-international.html?cid=20077160&cov=NA&heading=11421203&searchpos=2237
171
/profile/10042535/wyrwas-aluminum-industries.html?cid=10042535&cov=NA&heading=11421203&searchpos=2238
172
/profile/10046131/xri-testing-div-xray-industries-inc.html?cid=10046131&cov=NA&heading=11421203&searchpos=2244
173
/profile/00394615/hubon-aluminum-foundry-inc.html?cid=394615&cov=NA&heading=11421203&searchpos=2227
174
/profile/00025333/cast-products-company-inc.html?cid=25333&cov=NA&heading=11421203&searchpos=2243
175
/profile/30416308/santiam-bronze-works.html?cid=30416308&cov=NA&heading=11421203&searchpos=2245
176
/profile/01237624/f-h-aluminum-inc.html?cid=1237624&cov=NA&heading=11421203&searchpos=2248
177
/profile/00102971/dillon-pattern-works-foundry-inc.html?cid=102971

Supplier

In [10]:
Supplier

,Name,Address,Manufacturer_state,Number,URL,Employees,Description,Casting_capabilities,Capabilities_Overall,Capabilities_Detailed
0,"Custom Die Casting Co., Inc.","Richmond, IN 47374",IN,765-935-3979,[],10-49,Company Description by Thomasnet\nManufacturer...,"[Castings, Castings: Die]",[Casting Services],[Casting ServicesCastingsCastings: Die]
1,Wynn Danzur Marketing,"Piqua, OH 45356",OH,812-750-0124 (fax),[],1-9,,"[Castings, Castings: Aluminum, Castings: Brass...",[Casting Services],[Casting ServicesCastingsCastings: AluminumCas...
2,Alloy-Met,"Feasterville, PA 19053",PA,215-355-1110,[],10-49,Company Description by Thomasnet\nTechnical ma...,"[Castings, Castings: Aluminum, Castings: Alumi...","[Casting Services, Consulting Services, Forgin...",[Casting ServicesCastingsCastings: AluminumCas...
3,RPM Engineering,"Ponte Vedra Beach, FL 32004",FL,904-285-4940,[],10-49,Company Description by Thomasnet\nManufacturer...,"[Castings, Castings: Alloy, Castings: Alloy St...","[Assemblies, Bonding Services, Casting Service...","[AssembliesPlastic Assemblies, Bonding Service..."
4,"Dillon Precision, Inc.","Shingle Springs, CA 95682",CA,530-672-6755 (fax),[],Not Available,Company Description by Thomasnet\nCustom manuf...,"[Castings, Castings: Machined]","[Casting Services, Machining, Milling Services...","[Casting ServicesCastingsCastings: Machined, M..."
5,Wehner Foundry,"Kansas City, MO 64127-2228",MO,816-241-7350,[],1-9,Company Description by Thomasnet\nManufacturer...,"[Castings, Castings: Brass, Castings: Bronze]",[Casting Services],[Casting ServicesCastingsCastings: BrassCastin...
6,Westchester Casting Corp.,"Mount Vernon, NY 10553",NY,914-664-3505,[],10-49,Company Description by Thomasnet\nManufacturer...,"[Castings, Castings: Alloy, Castings: Aluminum...","[Casting Services, Traps, Valves]",[Casting ServicesCastingsCastings: AlloyCastin...
7,O'Keefe Casting Co.,"Lorain, OH 44055",OH,440-277-5427,[],10-49,Company Description by Thomasnet\nManufacturer...,"[Castings, Castings: Aluminum, Castings: Non-F...",[Casting Services],[Casting ServicesCastingsCastings: AluminumCas...
8,Non-Ferrous Heat Treating,"Cleveland, OH 44137",OH,216-662-0800,[],10-49,Company Description by Thomasnet\nHeat Treatin...,"[Castings, Castings: Aluminum, Castings: Non-F...","[Aluminum, Baths, Casting Services, Fabricatio...",[AluminumAlloys: AluminumAluminumExtrusions: M...
9,Scot Hone Corp.,"Lone Star, TX 75668",TX,903-639-2551,[],Not Available,Company Description by Thomasnet\nHone Tooling\n,"[Castings, Castings: Centrifugally Cast, Casti...","[Aluminum, Bars, Boring Services, Brass, Bronz...",[AluminumAlloy Bars & TubesAlloysAlloys: Alumi...
